In [ ]:
import numpy as np, pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
import nltk
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
X = df["Phrase"].tolist()
Y = df["Sentiment"].apply(lambda i: 0 if i <= 2 else 1)
Y.value_counts()

# Preprocess Texts

In [ ]:
lemmatizer = WordNetLemmatizer()
def proc_text(messy): #input is a single string
    first = BeautifulSoup(messy, "lxml").get_text() #gets text without tags or markup, remove html
    second = re.sub("[^a-zA-Z]"," ",first) #obtain only letters
    third = second.lower().split() #obtains a list of words in lower case
    fourth = set([lemmatizer.lemmatize(str(x)) for x in third]) #lemmatizing
    stops = set(stopwords.words("english")) #faster to search through a set than a list
    almost = [w for w in fourth if not w in stops] #remove stop words
    final = " ".join(almost)
    return final

In [ ]:
X = [proc_text(i) for i in X]

# split train test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=100, test_size=0.2, stratify=Y)
print("Training Set has {} Positive Labels and {} Negative Labels".format(sum(y_train), len(y_train) - sum(y_train)))
print("Test Set has {} Positive Labels and {} Negative Labels".format(sum(y_test), len(y_test) - sum(y_test)))

# train model

In [ ]:
#Features will be built using tfidf.
pos_weights = (len(y_train) - sum(y_train)) / (sum(y_train)) 
pipeline_tf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('classifier', DecisionTreeClassifier(random_state=100, class_weight={0: 1, 1: pos_weights}))
])

In [ ]:
pipeline_tf.fit(X_train, y_train)

# Evaluate Model (Before Thresholding)

In [ ]:
predictions = pipeline_tf.predict(X_test)
predicted_proba = pipeline_tf.predict_proba(X_test)

In [ ]:
print("Accuracy Score Before Thresholding: {}".format(accuracy_score(y_test, predictions)))
print("Precision Score Before Thresholding: {}".format(precision_score(y_test, predictions)))
print("Recall Score Before Thresholding: {}".format(recall_score(y_test, predictions)))
print("F1 Score Before Thresholding: {}".format(f1_score(y_test, predictions)))
print("ROC AUC Score: {}".format(roc_auc_score(y_test, predicted_proba[:, -1])))

## Confusion Matrix of Model (Before Thresholding) 

In [ ]:
y_actual = pd.Series(y_test, name='Actual')
y_predict_tf = pd.Series(predictions, name='Predicted')
df_confusion = pd.crosstab(y_actual, y_predict_tf, rownames=['Actual'], colnames=['Predicted'], margins=True)
print(df_confusion)

## ROC Curve

In [ ]:
false_pos_rate, true_pos_rate, proba = roc_curve(y_test, predicted_proba[:, -1])
plt.figure()
plt.plot([0,1], [0,1], linestyle="--") # plot random curve
plt.plot(false_pos_rate, true_pos_rate, marker=".", label=f"AUC = {roc_auc_score(y_test, predicted_proba[:, -1])}")
plt.title("ROC Curve")
plt.ylabel("True Positive Rate")
plt.xlabel("False Positive Rate")
plt.legend(loc="lower right")

# Obtain Optimal Probability Thresholds with ROC Curve 

In [ ]:
'''
The curve is plots values of true positive rates (y-axis) against those of false positive rates (x-axis) 
and these values are plotted at various probability thresholds.
There can be two ways of obtaining a more optimal probability threshold for the positive class:

##Youden's J Statistic
Find the maximum difference between true positive rate and false positive rate 
and the probability threshold tied tagged to this largest difference would be 
the selected one

Its computed as
J = TruePositiveRate + TrueNegativeRate − 1
  = TruePositiveRate − FalsePositiveRate


##Euclidean Distance
The most optimal ROC curve would be one that leans towards the top left of the plot, 
i.e. true positive rate of 1 and false positive rate of 0.
Select the probability threshold as the most optimal one if its true positive rate 
and false positive rate are closest to the ones mentioned in the previous point 
in terms of Euclidean distance, i.e.

d(tpr,fpr)=[(tpr1−tpr2)sq + (fpr1−fpr2)sq]sq_root

In [ ]:
#using the Youden's J statistic to obtain the optimal probability threshold 
#and this method gives equal weights to both false positives and false negatives
optimal_proba_cutoff = sorted(list(zip(np.abs(true_pos_rate - false_pos_rate), proba)), key=lambda i: i[0], reverse=True)[0][1]
roc_predictions = [1 if i >= optimal_proba_cutoff else 0 for i in predicted_proba[:, -1]]

## Evaluate Model (After Thresholding)

In [ ]:
print("Accuracy Score Before and After Thresholding: {}, {}".format(accuracy_score(y_test, predictions), accuracy_score(y_test, roc_predictions)))
print("Precision Score Before and After Thresholding: {}, {}".format(precision_score(y_test, predictions), precision_score(y_test, roc_predictions)))
print("Recall Score Before and After Thresholding: {}, {}".format(recall_score(y_test, predictions), recall_score(y_test, roc_predictions)))
print("F1 Score Before and After Thresholding: {}, {}".format(f1_score(y_test, predictions), f1_score(y_test, roc_predictions)))

## Confusion Matrix of Model (After Thresholding)

In [ ]:
y_actual = pd.Series(y_test, name='Actual')
y_predict_tf = pd.Series(roc_predictions, name='Predicted')
df_confusion = pd.crosstab(y_actual, y_predict_tf, rownames=['Actual'], colnames=['Predicted'], margins=True)
print (df_confusion)